# EC2 인스턴스 타입 변경

- 클라우드 컴퓨팅의 장점을 최대한 이용하여 프로비전 이후에도 적절한 타입/사이즈 변경 가능
- 두 가지 변경
    - 사이즈 변경 : 사용량에 맞춰 같은 인스턴스 패밀리 안의 다른 인스턴스로 변경 (예 : t3.micro -> t3.medium)
    - 패밀리 변경 : 워크로드의 성격에 따라 다른 인스턴스 패밀리로 변경 (예 : t3.micro -> m5.large)
- 호환성 체크 필요
    - 가상화 타입, 아키텍처, EBS 볼륨 개수 등
- 인스턴스 정지 필요
    - Spot 인스턴스는 변경 불가능
    - Health Check 불가능 -> Autoscaling에 영향 (오토 스케일링 수정 필요)
- 참고 사항
    - 인스턴스 아이디는 유지
    - Private IP는 유지하지만 Public IP는 변경(Elastic IP의 경우 고정)
- 인스턴스 스토어와 EBS 사용 인스턴스에 따라 과정이 달라짐
    - 인스턴스 스토어의 경우 영구적인 스토리지로 백업 필요

## Root 볼륨의 호환 여부에 따른 변경 방법
    - 호환되는 EBS의 경우 : 그냥 변경
    - 비호환 EBS : 새로운 인스턴스 프로비젼
    - 인스턴스 스토어 : 새로운 인스턴스 프로비전
    - 구체적인 과정
        1. 호환되는 EBS : 바로 인스턴스 타입 변경
        2. 호환되지 않는 EBS : EBS 분리, 신규 인스턴스 생성, 기존 EBS Attach
        3. 호환되지 않는 EBS(2) : EBS 분리, 스냅샷 생성, AMI 생성, AMI를 통해 신규 인스턴스 생성
        4. 인스턴스 스토어 : 인스턴스 스토어를 EBS로 백업.

## DEMO - 인스턴스 타입 변경
1. 인스턴스 생성
    - 키페어 x, 보안그룹 디폴트
2. 인스턴스 중지
    - 인스턴스 설정 - 인스턴스 유형 변경
    - 아키텍처 확인 및 변경 (t2.micro -> t2.medium)
    - 퍼블릭 주소 변경 확인 (프라이빗 주소는 그대로)
3. 인스턴스 다시 중지
    - 인스턴스 설정 - 인스턴스 유형 변경
    - 아키텍처 확인 및 변경 (t2.medium -> m3g.medium)
    - 아키텍처가 다르기 때문에 변경 불가능
4. 스냅샷으로 새로운 인스턴스 프로비전
    - 기존 인스턴스의 스토리지 메뉴 선택
    - 볼륨 ID 선택 - 작업에서 스냅샷 생성
    - 스냅샷에서 이미지 생성
        - 아키텍처를 잘 골라야 함 (m3g.medium의 arm64 선택)
    - AMI로 인스턴스 시작
5. 기존 EBS를 분리한 뒤, 신규 인스턴스에 Attach
    - 인스턴스 종료 시 EBS 볼륨 삭제는 초기 콘솔에서만 설정 가능하고, 일단 프로비전된 인스턴스는 콘솔에서 옵션 변경이 불가능하다.
    - 그래서 CLI를 이용해야 함. Cloudshell에 다음 명령어 입력
        - aws ec2 modify-instance-attribute --instance-id {인스턴스_아이디} --block-device-mappings "[{\"DeviceName\":\"/dev/xvda\",\"Ebs\":{\"DeleteOnTermination\":false}}]"
        - 기존 인스턴스의 스토리지 메뉴를 보면, EBS 종료시 삭제 옵션이 아니오로 바뀐걸 확인할 수 있다.
    - 기존 인스턴스 종료 (EBS는 살아있음)
    - 새로운 인스턴스에 Attach, 다만 AZ가 다른 경우 Attach가 불가능!!
        - EBS의 가용 영역을 미리 확인해두고, 새로운 인스턴스를 프로비전할 때 같은 AZ를 고르자!
        - 이제 EBS의 볼륨 연결이 가능!
6. 인스턴스, AMI, 스냅샷 삭제, EBS 볼륨도(종료 후 삭제 꺼진) 삭제